In [13]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [14]:
train_directory = "/content/drive/MyDrive/Datathon/train"
training_datagen = ImageDataGenerator(rescale=1./255,
zoom_range=0.15,
horizontal_flip=True,
fill_mode='nearest')


valid_directory = "/content/drive/MyDrive/Datathon/dev"
validation_datagen = ImageDataGenerator(rescale = 1./255)


train_generator = training_datagen.flow_from_directory(
train_directory,
target_size=(224,224),
shuffle = True,
class_mode='categorical',
batch_size = 64)

validation_generator = validation_datagen.flow_from_directory(
	valid_directory,
	target_size=(224,224),
	class_mode='categorical',
  batch_size= 16
)


Found 2094 images belonging to 2 classes.
Found 524 images belonging to 2 classes.


In [15]:
from tensorflow.keras.optimizers import RMSprop,Adam
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(224, 224, 3)),
          tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
          tf.keras.layers.Conv2D(256, (5,5), activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
          tf.keras.layers.Conv2D(384, (5,5), activation='relu'),
          tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(2048, activation='relu'),
          tf.keras.layers.Dropout(0.25),
          tf.keras.layers.Dense(1024, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 6, 384)         2457984   
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 2, 2, 384)       

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
history = model.fit(
    train_generator,
    steps_per_epoch = 15,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 15
    #callbacks=[callbacks]
)

Epoch 1/50
15/15 [==============================] - 22s 1s/step - loss: 0.5700 - acc: 0.6906 - val_loss: 0.2876 - val_acc: 0.8917
Epoch 2/50
15/15 [==============================] - 20s 1s/step - loss: 0.3976 - acc: 0.8259 - val_loss: 0.2784 - val_acc: 0.8750
Epoch 3/50
15/15 [==============================] - 21s 1s/step - loss: 0.2853 - acc: 0.8729 - val_loss: 0.1989 - val_acc: 0.9250
Epoch 4/50
15/15 [==============================] - 22s 1s/step - loss: 0.2753 - acc: 0.8917 - val_loss: 0.1974 - val_acc: 0.9125
Epoch 5/50
15/15 [==============================] - 21s 1s/step - loss: 0.2373 - acc: 0.9073 - val_loss: 0.3611 - val_acc: 0.8500
Epoch 6/50
15/15 [==============================] - 22s 1s/step - loss: 0.2786 - acc: 0.8854 - val_loss: 0.1551 - val_acc: 0.9458
Epoch 7/50
15/15 [==============================] - 21s 1s/step - loss: 0.2503 - acc: 0.9055 - val_loss: 0.1835 - val_acc: 0.9417
Epoch 8/50
15/15 [==============================] - 20s 1s/step - loss: 0.2127 - acc: 0.91

In [7]:
model.save('/content/drive/MyDrive/Datathon/100узщср/100epoch')


In [18]:
from tensorflow.keras.utils import load_img, img_to_array 
import numpy as np
from tensorflow.keras.models import load_model
path = '/content/drive/MyDrive/Datathon/experiment/578.jpg'
img = load_img(path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0) /255

saved_model = load_model('/content/drive/MyDrive/Datathon/models/')
classes = model.predict(x)
print(np.argmax(classes[0])==0, max(classes[0]))

1/1 [==============================] - 0s 15ms/step
True 0.92282605


In [ ]:
#writing to the test_labels.csv

import os
from tensorflow.keras.utils import load_img, img_to_array 
import numpy as np
from tensorflow.keras.models import load_model
import csv

f = open('/content/drive/MyDrive/Datathon/test_labels.csv', 'w')
saved_model = load_model('/content/drive/MyDrive/Datathon/100узщср/100epoch')
path = '/content/drive/MyDrive/Datathon/test_imgs/'
for i in range(1,1000):
  cur = f'{path}{i}.jpg'
  if os.path.isfile(cur) == 0:
    i+=1
    continue
  img = load_img(cur, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0) /255
  classes = saved_model.predict(x)
  data = [f'{i}.jpg', 1-np.argmax(classes[0])]

  writer = csv.writer(f)
  writer.writerow(data)
  print(f'{i}.jpg')
  print(1-np.argmax(classes[0]))